<a href="https://colab.research.google.com/github/Vamsikrishna1804/Vamsikrishnabharghava_INFO5731_Spring2021/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [11]:
# Write your code here
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import pandas as pd
import nltk
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, KFold, cross_val_score
from xgboost import XGBClassifier


In [22]:
train_df = pd.read_csv(r'/content/stsa-train.txt',sep = 'delimiter=',header= None,names=['Text'])
train_df[['Sentiment','Text']] = train_df["Text"].str.split(" ", 1, expand=True)
test_df = pd.read_csv(r'/content/stsa-test.txt',sep = 'delimiter=',header= None,names=['Text'])
test_df[['Sentiment','Text']] = test_df["Text"].str.split(" ", 1, expand=True)
train_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,Text,Sentiment
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [23]:
test_df.head()

,Text,Sentiment
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [24]:
# Clean data
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()
def clean_text(data):
  data="".join([word.lower() for word in data if word not in string.punctuation])
  data = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", data)
  tokens = re.split('\W+',data)
  data = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:

# Convert text and train data into numerical
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train_df['Text'])
print(X_tfidf.shape)
X_tfidf_df=pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns=tfidf_vect.get_feature_names()
X_test_tfidf = tfidf_vect.transform(test_df['Text'])
print(X_test_tfidf.shape)

(6920, 13343)
(1821, 13343)


In [26]:
#Algorithms
#(1) MultinominalNB
mnb = MultinomialNB()
#(2) SVM
svm = LinearSVC()
#(3) KNN
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
#(4) Decision tree
dt = DecisionTreeClassifier()
#(5) Random Forest
rf = RandomForestClassifier()
#(6) XGBoost
xgb = XGBClassifier()

#split the data
x_train, x_test, y_train, y_test = train_test_split(X_tfidf_df, train_df['Sentiment'].values,
                                                test_size=0.2, random_state=42)

In [14]:
#Defining method for cross validation score for all models
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
def cross_validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val


In [37]:
#(1) MultinominalNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
mnb_test = mnb.fit(x_train,y_train)
y_mnb = mnb_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_mnb,y_test))
print(classification_report(y_test,y_mnb))
from sklearn.model_selection import cross_val_score
scores_MNB = cross_val_score(mnb, x_test, y_test, cv=10)
kfold = KFold(10, random_state = 7, shuffle=True)
print("Accuracy with MNB",scores_MNB.mean())
mnb_scores_mean = cross_validation_score(mnb,x_test,y_test)
print('MNB Cross Validation Score is {0}'.format(mnb_scores_mean))
test_predicted_mnb = mnb.predict(x_test)
test_accuracy_mnb = round(accuracy_score(test_predicted_mnb, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_mnb)
print(classification_report(y_test, test_predicted_mnb))


Accuracy Score 0.7955202312138728
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       671
           1       0.76      0.88      0.82       713

    accuracy                           0.80      1384
   macro avg       0.80      0.79      0.79      1384
weighted avg       0.80      0.80      0.79      1384

Accuracy with MNB 0.7247054530288813
MNB Cross Validation Score is 0.7138828067980398
Testing Data
Accuracy -  79.55
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       671
           1       0.76      0.88      0.82       713

    accuracy                           0.80      1384
   macro avg       0.80      0.79      0.79      1384
weighted avg       0.80      0.80      0.79      1384



In [36]:
#(2) SVM
svm_test = svm.fit(x_train,y_train)
y_svm = svm_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_svm,y_test))
print(classification_report(y_test,y_svm))
from sklearn.model_selection import cross_val_score
scores_SVM = cross_val_score(svm, x_test, y_test, cv=10)
print("Accuracy with SVM",scores_SVM.mean())
svm_scores_mean = cross_validation_score(svm, x_test,y_test)
print('SVM Cross Validation Score is {0}'.format(svm_scores_mean))
test_predicted_svm = svm.predict(x_test)
test_accuracy_svm = round(accuracy_score(test_predicted_svm, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_svm)
print(classification_report(y_test, test_predicted_svm))


Accuracy Score 0.791907514450867
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       671
           1       0.78      0.83      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384

Accuracy with SVM 0.7348034615785632
SVM Cross Validation Score is 0.7348451673443853
Testing Data
Accuracy -  79.19
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       671
           1       0.78      0.83      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384



In [42]:
#KNN Model
knn_test = knn.fit(x_train,y_train)
y_knn = knn_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_knn,y_test))
print(classification_report(y_test,y_knn))
from sklearn.model_selection import cross_val_score
scores_KNN = cross_val_score(knn, x_test, y_test, cv=10)
print("Accuracy with knn",scores_KNN.mean())
knn_scores_mean = cross_validation_score(knn, x_test,y_test)
print('KNN Cross Validation Score is {0}'.format(knn_scores_mean))
test_predicted_knn = knn.predict(x_test)
test_accuracy_knn = round(accuracy_score(test_predicted_knn, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_knn)
print(classification_report(y_test, test_predicted_knn))

Accuracy Score 0.7398843930635838
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       671
           1       0.74      0.77      0.75       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384

Accuracy with knn 0.6675737670732979
KNN Cross Validation Score is 0.6669325409237826
Testing Data
Accuracy -  73.99
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       671
           1       0.74      0.77      0.75       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384



In [41]:
#(4) Decision tree
des_tree_test = dt.fit(x_train,y_train)
y_des_tree = des_tree_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_des_tree,y_test))
print(classification_report(y_test,y_des_tree))
scores_DT = cross_val_score(dt, x_test, y_test, cv=10)
print("Accuracy with Decision trees",scores_DT.mean())
dt_scores_mean = cross_validation_score(dt, x_test,y_test)
print('Decision Tree Cross Validation Score is {0}'.format(dt_scores_mean))
test_predicted_dt = dt.predict(x_test)
test_accuracy_dt = round(accuracy_score(test_predicted_dt, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_dt)
print(classification_report(y_test, test_predicted_dt))

Accuracy Score 0.6596820809248555
              precision    recall  f1-score   support

           0       0.66      0.61      0.64       671
           1       0.66      0.70      0.68       713

    accuracy                           0.66      1384
   macro avg       0.66      0.66      0.66      1384
weighted avg       0.66      0.66      0.66      1384

Accuracy with Decision trees 0.6191429465123554
Decision Tree Cross Validation Score is 0.6105150662079032
Testing Data
Accuracy -  65.97
              precision    recall  f1-score   support

           0       0.66      0.61      0.64       671
           1       0.66      0.70      0.68       713

    accuracy                           0.66      1384
   macro avg       0.66      0.66      0.66      1384
weighted avg       0.66      0.66      0.66      1384



In [43]:

#(5) Random Forest
rf_test = rf.fit(x_train,y_train)
y_rf = rf_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_rf,y_test))
print(classification_report(y_test,y_rf))
scores_RF = cross_val_score(rf, x_test, y_test, cv=10)
print("Accuracy with Random Forest",scores_RF.mean())
rf_scores_mean = cross_validation_score(rf, x_test,y_test)
print('Random Forest Cross Validation Score is {0}'.format(rf_scores_mean))
test_predicted_rf = rf.predict(x_test)
test_accuracy_rf = round(accuracy_score(test_predicted_rf, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_rf)
print(classification_report(y_test, test_predicted_rf))

Accuracy Score 0.7398843930635838
              precision    recall  f1-score   support

           0       0.78      0.65      0.71       671
           1       0.71      0.82      0.77       713

    accuracy                           0.74      1384
   macro avg       0.75      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384

Accuracy with Random Forest 0.6791419038682097
Random Forest Cross Validation Score is 0.6740955062037326
Testing Data
Accuracy -  73.99
              precision    recall  f1-score   support

           0       0.78      0.65      0.71       671
           1       0.71      0.82      0.77       713

    accuracy                           0.74      1384
   macro avg       0.75      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384



In [44]:
#(6) XGBoost
xgb_test = xgb.fit(x_train,y_train)
y_xgb = xgb_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_xgb,y_test))
print(classification_report(y_test,y_xgb))
scores_XGB = cross_val_score(xgb, x_test, y_test, cv=10)
print("Accuracy with XGBoost",scores_XGB.mean())
xgb_scores_mean = cross_validation_score(xgb, x_test,y_test)
print('XGBoost Cross Validation Score is {0}'.format(xgb_scores_mean))
test_predicted_xgb = xgb.predict(x_test)
test_accuracy_xgb = round(accuracy_score(test_predicted_xgb, y_test),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_xgb)
print(classification_report(y_test, test_predicted_xgb))

Accuracy Score 0.6445086705202312
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       671
           1       0.61      0.88      0.72       713

    accuracy                           0.64      1384
   macro avg       0.68      0.64      0.62      1384
weighted avg       0.68      0.64      0.62      1384

Accuracy with XGBoost 0.6184704410384736
XGBoost Cross Validation Score is 0.614211239703889
Testing Data
Accuracy -  64.45
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       671
           1       0.61      0.88      0.72       713

    accuracy                           0.64      1384
   macro avg       0.68      0.64      0.62      1384
weighted avg       0.68      0.64      0.62      1384



In [45]:
print("Accuracy with MNB",scores_MNB.mean())
print("Accuracy with SVM",scores_SVM.mean())
print("Accuracy with knn",scores_KNN.mean())
print("Accuracy with Decision trees",scores_DT.mean())
print("Accuracy with Random Forest",scores_RF.mean())
print("Accuracy with XGBoost",scores_XGB.mean())

Accuracy with MNB 0.7247054530288813
Accuracy with SVM 0.7348034615785632
Accuracy with knn 0.6675737670732979
Accuracy with Decision trees 0.6191429465123554
Accuracy with Random Forest 0.6791419038682097
Accuracy with XGBoost 0.6184704410384736


In [49]:
predict_svm = svm_test.predict(X_test_tfidf)
print('Final trained model(SVM) with high accuracy evaluated on the test data: %s' % accuracy_score(predict_svm,test_df['Sentiment']))

Final trained model(SVM) with high accuracy evaluated on the test data: 0.7891268533772653
